In [3]:
import orca
import warnings

import urbansim_templates
import os; os.chdir('../')

from scripts import models, datasources, variables

warnings.simplefilter('ignore')

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [4]:
import pandas as pd
import numpy as np

### Generate accessibility variables (or load them if they already exist)

Initialize networks

In [5]:
orca.run(['initialize_network_small', 'initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


If they exist already:

In [6]:
walk_net_vars = pd.read_csv('../ual_model_workspace/fall-2018-models/data/walk_net_vars.csv', index_col='osmid')
drive_net_vars = pd.read_csv('../ual_model_workspace/fall-2018-models/data/drive_net_vars.csv', index_col='osmid')
orca.add_table('nodeswalk', walk_net_vars)
orca.add_table('nodessmall', drive_net_vars)

If generating on-the-fly:

In [7]:
# orca.run(['network_aggregations_small', 'network_aggregations_walk'])

### Simulate workplace location choices

Run the model step

In [8]:
orca.run(['wlcm_simulate'])

Running step 'wlcm_simulate'
Iteration 1: 191230 of 3060996 valid choices
Iteration 2: 381658 of 3060996 valid choices
Iteration 3: 571355 of 3060996 valid choices
Iteration 4: 760038 of 3060996 valid choices
Iteration 5: 947604 of 3060996 valid choices
Iteration 6: 1133729 of 3060996 valid choices
Iteration 7: 1317886 of 3060996 valid choices
Iteration 8: 1499993 of 3060996 valid choices
Iteration 9: 1679269 of 3060996 valid choices
Iteration 10: 1854852 of 3060996 valid choices
Iteration 11: 2024944 of 3060996 valid choices
Iteration 12: 2186665 of 3060996 valid choices
Iteration 13: 2334854 of 3060996 valid choices
Iteration 14: 2459038 of 3060996 valid choices
Iteration 15: 2540488 of 3060996 valid choices
Iteration 16: 2573911 of 3060996 valid choices
Iteration 17: 2578046 of 3060996 valid choices
No valid alternatives for the remaining choosers
Time to execute step 'wlcm_simulate': 236.07 s
Total time to execute iteration 1 with iteration value None: 236.07 s


Query the updated persons table to assess job placement

In [9]:
persons_w_jobs = orca.get_table('persons').to_frame()
num_persons_w_jobs = len(persons_w_jobs[~pd.isnull(persons_w_jobs['job_id'])])
wlcm = urbansim_templates.modelmanager.get_step('WLCM')
num_workers = len(persons_w_jobs.query(' and '.join(wlcm.out_chooser_filters)))

In [10]:
print("WLCM placed {0} of {1} workers ({2}%) during simulation.".format(
    num_persons_w_jobs, num_workers, np.round(num_persons_w_jobs/num_workers * 100, 1)))

WLCM placed 2578046 of 3060996 workers (84.2%) during simulation.


### Simulate time-of-day choices for mandatory trip (commute) departure times

#### a) Assign commuters to depature time chunks

In [11]:
orca.run(['TOD_choice_simulate'])

Running step 'TOD_choice_simulate'
Time to execute step 'TOD_choice_simulate': 227.02 s
Total time to execute iteration 1 with iteration value None: 227.02 s


#### b) Assign actual departure times based on known distributions

In [12]:
orca.run(['TOD_distribution_simulate'])

Running step 'TOD_distribution_simulate'
Time to execute step 'TOD_distribution_simulate': 611.60 s
Total time to execute iteration 1 with iteration value None: 611.60 s


Query the updated persons table to assess TOD chunk assignment

In [13]:
persons_w_TOD = orca.get_table('persons').to_frame()
num_persons_w_TOD = len(persons_w_TOD[~pd.isnull(persons_w_TOD['HW_ST'])])

In [14]:
print("Departure time assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_TOD, num_workers, np.round(num_persons_w_TOD/num_persons_w_jobs * 100, 1)))

Departure time assigned to 2502430 of 3060996 workers with jobs (97.1%) during simulation.


### Simulate auto-ownership choices

In [15]:
m = urbansim_templates.modelmanager.get_step('auto_ownership')

In [16]:
m.filters = None
m.out_table = 'households'
m.out_column = 'cars_alt'

In [17]:
df = m._get_data('predict')

KeyError: 'table not found: hh_merged'

In [ ]:
df.head()

In [ ]:
long_df = m._to_long(df, 'predict')

In [ ]:
long_df.head()

In [ ]:
num_obs = len(df)

In [ ]:
num_alts = len(m._get_alts())

In [ ]:
m.model

In [ ]:
probs = m.model.predict(long_df)

In [ ]:
orca.run(['auto_ownership_simulate'])

### Simulate primary commute mode choice

In [ ]:
# orca.run(['primary_mode_choice'])

### Generate activity plans

Activity plans currently written according to [BEAM](http://beam.lbl.gov/) specification

In [ ]:
orca.run(['generate_activity_plans'])

Load activity plan data and investigate it

In [ ]:
activities = pd.read_csv('./data/urbansim_beam_plans.csv')

In [ ]:
activities.head()